In [1]:

import tensorflow as tf

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

In [3]:
X = tf.placeholder(tf.float32, name = "X")
label = tf.placeholder(tf.float32, name = "label")

In [7]:
def compute_loss(l_pred_w,class_loss_non_reducted, reduction='mean', margin=1.0):

    # compute the classification loss non reducted
    get_batch_size = tf.shape(class_loss_non_reducted)[0]


    with tf.compat.v1.variable_scope("Reference_Loss_LossNet"):
        l_true = (class_loss_non_reducted - class_loss_non_reducted[::-1] )[:class_loss_non_reducted.shape[0]//2]
        # get the value without the gradient
        l_true = tf.stop_gradient(l_true)


    # value used in the lossnet loss
    one = (2 * tf.math.sign(  tf.clip_by_value( l_true, 0, 1))) - 1


    with tf.compat.v1.variable_scope("Learning_loss_loss_whole"):
        if reduction == 'mean':
            l_loss_w = tf.reduce_sum(tf.clip_by_value(margin - one * l_pred_w, 0,10000))
            l_loss_w = tf.math.divide(l_loss_w , tf.cast(tf.shape(l_pred_w)[0], l_loss_w.dtype) ) # Note that the size of l_pred is already halved
        elif reduction == 'none':
            l_loss_w = tf.clip_by_value(margin - one * l_pred_w, 0,10000)
        else:
            NotImplementedError()

    with tf.compat.v1.variable_scope("Learning_loss_loss_split"):
        if reduction == 'mean':
            l_loss_s = tf.reduce_sum(tf.clip_by_value(margin - one * l_pred_w, 0,10000))
            l_loss_s = tf.math.divide(l_loss_s , tf.cast(tf.shape(l_pred_w)[0], l_loss_s.dtype) ) # Note that the size of l_pred is already halved
        elif reduction == 'none':
            l_loss_s = tf.clip_by_value(margin - one * l_pred_w, 0,10000)
        else:
            NotImplementedError()

    return l_loss_w, l_loss_s

In [ ]:
def LossPredLoss(x, target, margin=1.0, reduction='mean'):
    assert len(x) % 2 == 0, 'the batch size is not even.'
    assert x.shape == x.flip(0).shape
    
    x = (x - x.flip(0))[:len(x)//2] # [l_1 - l_2B, l_2 - l_2B-1, ... , l_B - l_B+1], where batch_size = 2B
    target = (target - target.flip(0))[:len(target)//2]
    target = target.detach()

    one = 2 * torch.sign(torch.clamp(target, min=0)) - 1 # 1 operation which is defined by the authors
    
    if reduction == 'mean':
        loss = torch.sum(torch.clamp(margin - one * x, min=0))
        loss = loss / x.size(0) # Note that the size of x is already halved
    elif reduction == 'none':
        loss = torch.clamp(margin - one * x, min=0)
    else:
        NotImplementedError()
    
    return loss

In [107]:
def compute_loss(x,target, reduction='mean', margin=1.0):

    # compute the classification loss non reducted
    get_batch_size = tf.shape(target)[0]
    
    
    x = (x - x[::-1])[:get_batch_size//2]
    target = (target - target[::-1] )[:get_batch_size//2]
    target = tf.stop_gradient(target)

    # value used in the lossnet loss
    one = (2 * tf.math.sign(tf.clip_by_value( target, 0, 1))) - 1

    if reduction == 'mean':
        loss = tf.reduce_sum(tf.clip_by_value(margin - one * x, 0,10000))
        loss = tf.math.divide(loss , tf.cast(get_batch_size//2, loss.dtype) ) # Note that the size of l_pred is already halved
    elif reduction == 'none':
        loss = tf.clip_by_value(margin - one * x, 0,10000)
    else:
        NotImplementedError()



    return loss

In [ ]:
def compute_loss(self,c_true):

    # compute the classification loss non reducted
    get_batch_size = tf.shape(c_true)[0]

    # Classification loss
    with tf.compat.v1.variable_scope("Classification_loss"):
        class_loss_non_reducted = tf.nn.softmax_cross_entropy_with_logits_v2(labels=c_true, logits=self.c_pred)
        c_loss = tf.reduce_mean(class_loss_non_reducted)

    with tf.compat.v1.variable_scope("Reference_Loss_LossNet"):
        l_true = (class_loss_non_reducted - class_loss_non_reducted[::-1] )[:get_batch_size//2]
        # get the value without the gradient
        l_true = tf.stop_gradient(l_true)
        
    
    # value used in the lossnet loss
    one = (2 * tf.math.sign(  tf.clip_by_value( l_true, 0, 1))) - 1

    with tf.compat.v1.variable_scope("Learning_loss_loss_whole"):
        
        l_pred_w = (self.l_pred_w - self.l_pred_w[::-1])[:get_batch_size//2]
        if self.reduction == 'mean':
            l_loss_w = tf.reduce_sum(tf.clip_by_value(self.margin - one * l_pred_w, 0,10000))
            l_loss_w = tf.math.divide(l_loss_w , tf.cast(tf.shape(l_pred_w)[0], l_loss_w.dtype) ) # Note that the size of l_pred is already halved
        elif self.reduction == 'none':
            l_loss_w = tf.clip_by_value(self.margin - one * self.l_pred_w, 0,10000)
        else:
            NotImplementedError()

    with tf.compat.v1.variable_scope("Learning_loss_loss_split"):
        l_pred_s = (self.l_pred_s - self.l_pred_s[::-1])[:get_batch_size//2]
        if self.reduction == 'mean':
            l_loss_s = tf.reduce_sum(tf.clip_by_value(self.margin - one * l_pred_s, 0,10000))
            l_loss_s = tf.math.divide(l_loss_s , tf.cast(tf.shape(self.l_pred_s)[0], l_loss_s.dtype) ) # Note that the size of l_pred is already halved
        elif self.reduction == 'none':
            l_loss_s = tf.clip_by_value(self.margin - one * self.l_pred_s, 0,10000)
        else:
            NotImplementedError()

    self.c_loss =  c_loss
    self.l_loss_w = l_loss_w
    self.l_loss_s = l_loss_s
    self.l_true  = l_true
    self.class_loss_non_reducted = class_loss_non_reducted

    return self.c_loss, self.l_loss_w, self.l_loss_s, self.l_true

In [108]:
res = compute_loss(X,label)

In [109]:
config_tf = tf.ConfigProto(allow_soft_placement=True) 
config_tf.gpu_options.allow_growth = True 
sess = tf.Session(config=config_tf)


In [117]:
import numpy as np

x = np.array([0.5023, 0.5100, 0.9195, 0.9535, 0.6252, 0.8706, 0.6801, 0.4186, 0.9509,0.3584, 0.5273, 0.9775, 0.4921, 0.4515, 0.7213, 0.8194])
target= np.array([0.0736, 0.0751, 0.8538, 0.7110, 0.9743, 0.7315, 0.1218, 0.6964, 0.4264, 0.4585, 0.3606, 0.0911, 0.9101, 0.0578, 0.1127, 0.6333])


x = np.array([1,2,3,4,5,6,7,8,9,10])
target= np.array([10,9,8,7,6,5,4,3,2,1])



In [118]:
result = sess.run(res, feed_dict={X:x, label:target})
print(result)

6.0


In [46]:
target_

array([[0.21069985, 0.90130608, 0.31085171, 0.83086515],
       [0.37410988, 0.70285848, 0.83830728, 0.55840195]])

In [49]:
temp = np.random.rand(16)

In [51]:
temp

array([0.80238206, 0.70935415, 0.99898735, 0.53499073, 0.42875013,
       0.46676256, 0.02110695, 0.77252365, 0.52069968, 0.47556882,
       0.26257111, 0.47124933, 0.803462  , 0.11010067, 0.98035664,
       0.21934148])

In [52]:
temp[::-1]

array([0.21934148, 0.98035664, 0.11010067, 0.803462  , 0.47124933,
       0.26257111, 0.47556882, 0.52069968, 0.77252365, 0.02110695,
       0.46676256, 0.42875013, 0.53499073, 0.99898735, 0.70935415,
       0.80238206])